In [2]:
#Importing Dependcies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [3]:
executable_path = {'executable_path': '/Users/sashadahir/Desktop/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
#Mars News Scraping

news_link = 'https://mars.nasa.gov/news/'
browser.visit(news_link)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [11]:
# Mars Space News Title and Paragraph Scraping

news_title = soup.find_all('div', class_='content_title')[1].text
print(mars_title)

print('*****************')
news_p = soup.find_all('div', class_='article_teaser_body')[1].text
print(news_p)   
    


5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover
*****************
NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel.


In [14]:
# image scraping

jpl_nasa_url = 'https://www.jpl.nasa.gov'
images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(images_url)

html = browser.html

images_soup = BeautifulSoup(html, 'html.parser')

In [19]:
# Retrieve featured image link
relative_image_path = images_soup.find_all('img')[3]["src"]
featured_image_url = jpl_nasa_url + relative_image_path
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24275-640x350.jpg
